In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.layers import multiply, Permute
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import load_model

start = dt.datetime.now()

def squeeze_excite_block(input, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

Using TensorFlow backend.


In [2]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Reshape
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import add
from keras.layers import multiply
from keras.regularizers import l2
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.applications.resnet50 import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras import backend as K



__all__ = ['SEResNet', 'SEResNet50', 'SEResNet101', 'SEResNet154', 'preprocess_input', 'decode_predictions']


WEIGHTS_PATH = ""
WEIGHTS_PATH_NO_TOP = ""


def SEResNet(input_shape=None,
             initial_conv_filters=64,
             depth=[3, 4, 6, 3],
             filters=[64, 128, 256, 512],
             width=1,
             bottleneck=False,
             weight_decay=1e-4,
             include_top=True,
             weights=None,
             input_tensor=None,
             pooling=None,
             classes=1000):
    """ Instantiate the Squeeze and Excite ResNet architecture. Note that ,
        when using TensorFlow for best performance you should set
        `image_data_format="channels_last"` in your Keras config
        at ~/.keras/keras.json.
        The model are compatible with both
        TensorFlow and Theano. The dimension ordering
        convention used by the model is the one
        specified in your Keras config file.
        # Arguments
            initial_conv_filters: number of features for the initial convolution
            depth: number or layers in the each block, defined as a list.
                ResNet-50  = [3, 4, 6, 3]
                ResNet-101 = [3, 6, 23, 3]
                ResNet-152 = [3, 8, 36, 3]
            filter: number of filters per block, defined as a list.
                filters = [64, 128, 256, 512
            width: width multiplier for the network (for Wide ResNets)
            bottleneck: adds a bottleneck conv to reduce computation
            weight_decay: weight decay (l2 norm)
            include_top: whether to include the fully-connected
                layer at the top of the network.
            weights: `None` (random initialization) or `imagenet` (trained
                on ImageNet)
            input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
                to use as image input for the model.
            input_shape: optional shape tuple, only to be specified
                if `include_top` is False (otherwise the input shape
                has to be `(224, 224, 3)` (with `tf` dim ordering)
                or `(3, 224, 224)` (with `th` dim ordering).
                It should have exactly 3 inputs channels,
                and width and height should be no smaller than 8.
                E.g. `(200, 200, 3)` would be one valid value.
            pooling: Optional pooling mode for feature extraction
                when `include_top` is `False`.
                - `None` means that the output of the model will be
                    the 4D tensor output of the
                    last convolutional layer.
                - `avg` means that global average pooling
                    will be applied to the output of the
                    last convolutional layer, and thus
                    the output of the model will be a 2D tensor.
                - `max` means that global max pooling will
                    be applied.
            classes: optional number of classes to classify images
                into, only to be specified if `include_top` is True, and
                if no `weights` argument is specified.
        # Returns
            A Keras model instance.
        """

    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')

    assert len(depth) == len(filters), "The length of filter increment list must match the length " \
                                       "of the depth list."

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      data_format=K.image_data_format(),
                                      require_flatten=False)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    x = _create_se_resnet(classes, img_input, include_top, initial_conv_filters,
                          filters, depth, width, bottleneck, weight_decay, pooling)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnext')

    # load weights

    return model


def SEResNet18(input_shape=None,
               width=1,
               bottleneck=False,
               weight_decay=1e-4,
               include_top=True,
               weights=None,
               input_tensor=None,
               pooling=None,
               classes=1000):
    return SEResNet(input_shape,
                    depth=[2, 2, 2, 2],
                    width=width,
                    bottleneck=bottleneck,
                    weight_decay=weight_decay,
                    include_top=include_top,
                    weights=weights,
                    input_tensor=input_tensor,
                    pooling=pooling,
                    classes=classes)


def SEResNet34(input_shape=None,
               width=1,
               bottleneck=False,
               weight_decay=1e-4,
               include_top=True,
               weights=None,
               input_tensor=None,
               pooling=None,
               classes=1000):
    return SEResNet(input_shape,
                    depth=[3, 4, 6, 3],
                    width=width,
                    bottleneck=bottleneck,
                    weight_decay=weight_decay,
                    include_top=include_top,
                    weights=weights,
                    input_tensor=input_tensor,
                    pooling=pooling,
                    classes=classes)


def SEResNet50(input_shape=None,
               width=1,
               bottleneck=True,
               weight_decay=1e-4,
               include_top=True,
               weights=None,
               input_tensor=None,
               pooling=None,
               classes=1000):
    return SEResNet(input_shape,
                    width=width,
                    bottleneck=bottleneck,
                    weight_decay=weight_decay,
                    include_top=include_top,
                    weights=weights,
                    input_tensor=input_tensor,
                    pooling=pooling,
                    classes=classes)


def SEResNet101(input_shape=None,
                width=1,
                bottleneck=True,
                weight_decay=1e-4,
                include_top=True,
                weights=None,
                input_tensor=None,
                pooling=None,
                classes=1000):
    return SEResNet(input_shape,
                    depth=[3, 6, 23, 3],
                    width=width,
                    bottleneck=bottleneck,
                    weight_decay=weight_decay,
                    include_top=include_top,
                    weights=weights,
                    input_tensor=input_tensor,
                    pooling=pooling,
                    classes=classes)


def SEResNet154(input_shape=None,
                width=1,
                bottleneck=True,
                weight_decay=1e-4,
                include_top=True,
                weights=None,
                input_tensor=None,
                pooling=None,
                classes=1000):
    return SEResNet(input_shape,
                    depth=[3, 8, 36, 3],
                    width=width,
                    bottleneck=bottleneck,
                    weight_decay=weight_decay,
                    include_top=include_top,
                    weights=weights,
                    input_tensor=input_tensor,
                    pooling=pooling,
                    classes=classes)


def _resnet_block(input, filters, k=1, strides=(1, 1)):
    ''' Adds a pre-activation resnet block without bottleneck layers
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
        strides: strides of the convolution layer
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    x = BatchNormalization(axis=channel_axis)(input)
    x = Activation('relu')(x)

    if strides != (1, 1) or init._keras_shape[channel_axis] != filters * k:
        init = Conv2D(filters * k, (1, 1), padding='same', kernel_initializer='he_normal',
                      use_bias=False, strides=strides)(x)

    x = Conv2D(filters * k, (3, 3), padding='same', kernel_initializer='he_normal',
               use_bias=False, strides=strides)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters * k, (3, 3), padding='same', kernel_initializer='he_normal',
               use_bias=False)(x)

    # squeeze and excite block
    x = squeeze_excite_block(x)

    m = add([x, init])
    return m


def _resnet_bottleneck_block(input, filters, k=1, strides=(1, 1)):
    ''' Adds a pre-activation resnet block with bottleneck layers
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
        strides: strides of the convolution layer
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    bottleneck_expand = 4

    x = BatchNormalization(axis=channel_axis)(input)
    x = Activation('relu')(x)

    if strides != (1, 1) or init._keras_shape[channel_axis] != bottleneck_expand * filters * k:
        init = Conv2D(bottleneck_expand * filters * k, (1, 1), padding='same', kernel_initializer='he_normal',
                      use_bias=False, strides=strides)(x)

    x = Conv2D(filters * k, (1, 1), padding='same', kernel_initializer='he_normal',
               use_bias=False)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters * k, (3, 3), padding='same', kernel_initializer='he_normal',
               use_bias=False, strides=strides)(x)
    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    x = Conv2D(bottleneck_expand * filters * k, (1, 1), padding='same', kernel_initializer='he_normal',
               use_bias=False)(x)

    # squeeze and excite block
    x = squeeze_excite_block(x)

    m = add([x, init])
    return m


def _create_se_resnet(classes, img_input, include_top, initial_conv_filters, filters,
                      depth, width, bottleneck, weight_decay, pooling):
    '''Creates a SE ResNet model with specified parameters
    Args:
        initial_conv_filters: number of features for the initial convolution
        include_top: Flag to include the last dense layer
        filters: number of filters per block, defined as a list.
            filters = [64, 128, 256, 512
        depth: number or layers in the each block, defined as a list.
            ResNet-50  = [3, 4, 6, 3]
            ResNet-101 = [3, 6, 23, 3]
            ResNet-152 = [3, 8, 36, 3]
        width: width multiplier for network (for Wide ResNet)
        bottleneck: adds a bottleneck conv to reduce computation
        weight_decay: weight_decay (l2 norm)
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
    Returns: a Keras Model
    '''
    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    N = list(depth)

    # block 1 (initial conv block)
    x = Conv2D(initial_conv_filters, (7, 7), padding='same', use_bias=False, strides=(2, 2),
               kernel_initializer='he_normal', kernel_regularizer=l2(weight_decay))(img_input)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # block 2 (projection block)
    for i in range(N[0]):
        if bottleneck:
            x = _resnet_bottleneck_block(x, filters[0], width)
        else:
            x = _resnet_block(x, filters[0], width)

    # block 3 - N
    for k in range(1, len(N)):
        if bottleneck:
            x = _resnet_bottleneck_block(x, filters[k], width, strides=(2, 2))
        else:
            x = _resnet_block(x, filters[k], width, strides=(2, 2))

        for i in range(N[k] - 1):
            if bottleneck:
                x = _resnet_bottleneck_block(x, filters[k], width)
            else:
                x = _resnet_block(x, filters[k], width)

    x = BatchNormalization(axis=channel_axis)(x)
    x = Activation('relu')(x)

    if include_top:
        x = GlobalAveragePooling2D()(x)
        x = Dense(classes, use_bias=False, kernel_regularizer=l2(weight_decay),
                  activation='softmax')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    return x

In [3]:
DP_DIR = '/home/k.vincent/'
INPUT_DIR = '/home/k.vincent/apm_file/'
BASE_SIZE = 256
NCSVS = 100
NCATS = 340
np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top3_acc( tgt, pred ):
    sc = np.mean( (pred[:,0]==tgt) | (pred[:,1]==tgt) | (pred[:,2]==tgt) )
    return sc

def custom_single_cnn(size):
    model = SEResNet34(input_shape=(size, size, 3), classes=340)
    
    return model

In [4]:
bi_factor = 4
STEPS = 1000
size = 128
batchsize = 512

model = custom_single_cnn(size=size)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
               metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   9408        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 32, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
activation

In [4]:
STEPS = 1000
size = 128
batchsize = 512


colors = [(255, 0, 0) , (255, 255, 0),  (128, 255, 0),  (0, 255, 0), (0, 255, 128), (0, 255, 255), 
          (0, 128, 255), (0, 0, 255), (128, 0, 255), (255, 0, 255)]
def draw_cv2(raw_strokes, size=256, lw=6):
    img = np.zeros((BASE_SIZE, BASE_SIZE,3), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = colors[min(t, len(colors)-1)]
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), color, lw, lineType=cv2.LINE_AA)
    if np.random.rand()>0.5:
        img = np.fliplr(img)
    if np.random.rand()>0.75:
        if np.random.rand()>0.50:
            img = img[ 4:, 4: ,:]
        else:
            img = img[ :-4, :-4 ,:]
    if np.random.rand()>0.50:
        img2 = cv2.resize(img, (200, 200))
        img = np.zeros((BASE_SIZE, BASE_SIZE,3), np.uint8)
        img[18:218,18:218, :] = img2

    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def image_generator(size, batchsize, ks, lw=6):
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(k))
            for df in pd.read_csv(filename, chunksize=batchsize):
                df['drawing'] = df['drawing'].apply(ast.literal_eval)
                x = np.zeros((len(df), size, size,3))
                for i, raw_strokes in enumerate(df.drawing.values):
                    #print(df.drawing.values)
                    #
                    x[i, :, :, :] = draw_cv2(raw_strokes, size=size, lw=lw)
                x = x / 255.
                x = x.reshape((len(df), size, size, 3)).astype(np.float32)
                y = keras.utils.to_categorical(df.y, num_classes=NCATS)
                yield x, y

def df_to_image_array(df, size, lw=6):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size,3))
    for i, raw_strokes in enumerate(df.drawing.values):
        #
        x[i, :, : ,:] = draw_cv2(raw_strokes, size=size, lw=lw)
    x = x / 255.
    x = x.reshape((len(df), size, size, 3)).astype(np.float32)
    return x

In [12]:
valid_df = pd.read_csv(os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(NCSVS - 1)), nrows=30000)
x_valid = df_to_image_array(valid_df, size)
y_valid = keras.utils.to_categorical(valid_df.y, num_classes=NCATS)
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(30000, 128, 128, 3) (30000, 340)
Validation array memory 5.49 GB


In [22]:
train_datagen = image_generator(size=size, batchsize=batchsize, ks=range(NCSVS - 1))
val_datagen = image_generator(size=size, batchsize=batchsize, ks=range(NCSVS - 1, NCSVS))

In [8]:
callbacks = [
    #EarlyStopping(monitor='val_top_3_accuracy', patience=15, min_delta=0.001, mode='max'),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5, min_delta=0.005, mode='max', cooldown=3),
    ModelCheckpoint("./black-white-7.model",monitor='val_top_3_accuracy', mode = 'max', save_best_only=True, verbose=1)
]
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=10, verbose=1,
    validation_data=val_datagen,validation_steps = 150,
    callbacks = callbacks
)

model.save('Autosaved_CNN')

Epoch 1/10
1000/1000 [==============================] - 1980s 2s/step - loss: 0.7903 - categorical_crossentropy: 0.7551 - categorical_accuracy: 0.8035 - top_3_accuracy: 0.9258 - val_loss: 0.7880 - val_categorical_crossentropy: 0.7528 - val_categorical_accuracy: 0.8032 - val_top_3_accuracy: 0.9263

Epoch 00001: val_top_3_accuracy improved from -inf to 0.92632, saving model to ./black-white-7.model
Epoch 2/10
1000/1000 [==============================] - 1984s 2s/step - loss: 0.7885 - categorical_crossentropy: 0.7533 - categorical_accuracy: 0.8041 - top_3_accuracy: 0.9258 - val_loss: 0.7807 - val_categorical_crossentropy: 0.7455 - val_categorical_accuracy: 0.8060 - val_top_3_accuracy: 0.9270

Epoch 00002: val_top_3_accuracy improved from 0.92632 to 0.92695, saving model to ./black-white-7.model
Epoch 3/10
1000/1000 [==============================] - 1992s 2s/step - loss: 0.7910 - categorical_crossentropy: 0.7559 - categorical_accuracy: 0.8040 - top_3_accuracy: 0.9254 - val_loss: 0.7867 - 

In [23]:
callbacks = [
    #EarlyStopping(monitor='val_top_3_accuracy', patience=15, min_delta=0.001, mode='max'),
    ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5, min_delta=0.005, mode='max', cooldown=3),
    ModelCheckpoint("./black-white-7.model",monitor='val_top_3_accuracy', mode = 'max', save_best_only=True, verbose=1)
]

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=3, verbose=1,
    validation_data=val_datagen,validation_steps = 150,
    callbacks = callbacks
)

model.save('Autosaved_CNN')

Epoch 1/3
1000/1000 [==============================] - 2065s 2s/step - loss: 0.7785 - categorical_crossentropy: 0.7437 - categorical_accuracy: 0.8060 - top_3_accuracy: 0.9273 - val_loss: 0.7847 - val_categorical_crossentropy: 0.7499 - val_categorical_accuracy: 0.8042 - val_top_3_accuracy: 0.9267

Epoch 00001: val_top_3_accuracy improved from -inf to 0.92669, saving model to ./black-white-7.model
Epoch 2/3
1000/1000 [==============================] - 2048s 2s/step - loss: 0.7755 - categorical_crossentropy: 0.7407 - categorical_accuracy: 0.8071 - top_3_accuracy: 0.9275 - val_loss: 0.7793 - val_categorical_crossentropy: 0.7445 - val_categorical_accuracy: 0.8057 - val_top_3_accuracy: 0.9283

Epoch 00002: val_top_3_accuracy improved from 0.92669 to 0.92832, saving model to ./black-white-7.model
Epoch 3/3
 759/1000 [=====================>........] - ETA: 7:36 - loss: 0.7548 - categorical_crossentropy: 0.7201 - categorical_accuracy: 0.8118 - top_3_accuracy: 0.9303

KeyboardInterrupt: 

## Create Submission

In [26]:
#If you have enough memory then ship this
#remove previous test data: 
try:
    del test
except:
    pass
try:
    del x_test
except:
    pass
try:
    del x_test2
except:
    pass
try:
    del test_predictions1
except:
    pass
try:
    del test_predictions2
except:
    pass
try:
    del test_predictions
except:
    pass
try:
    del top3
except:
    pass
try:
    del submission
except:
    pass

try:
    del top3cats
except:
    pass
try:
    del x_valid2
    del valid_predictions1
    del valid_predictions2
except:
    pass

#Remove training data. Do so when training 64*64 model
try:
    del x_valid
    del y_valid
    del valid_df
    del x
    pass
except:
    pass

In [ ]:
#Create Submission
#batched prediction due to memory constrain
INPUT_DIR = '/home/k.vincent/'
test = pd.read_csv(os.path.join(INPUT_DIR, 'test_simplified.csv'))

test_predictions1 = np.zeros(shape=(0,340))
test_predictions2 = np.zeros(shape=(0,340))

for index, ob in test.groupby(np.arange(len(test))//8000):
    x_test = df_to_image_array(ob, size)
    x_test2 = np.array( [ np.fliplr(x_test[i]) for i in range(x_test.shape[0])] )
    
    temp_pred = model.predict(x_test, batch_size=128, verbose=1)
    test_predictions1 = np.concatenate((test_predictions1, temp_pred))
    
    temp_pred = model.predict(x_test2, batch_size=128, verbose=1)
    test_predictions2 = np.concatenate((test_predictions2, temp_pred))

In [29]:
#Prediction ensembling and write file
test_predictions = 1*test_predictions1 + 0*test_predictions2
INPUT_DIR = '/home/k.vincent/apm_file/'
top3 = preds2catids(test_predictions)
cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('_submit1', index=False)
submission.head()
submission.shape

,a,b,c
0,radio,stereo,alarm_clock
1,hockey_puck,bottlecap,pool
2,The_Great_Wall_of_China,castle,fence
3,mountain,tent,The_Eiffel_Tower
4,campfire,fireplace,leaf


(112199, 3)

,key_id,word
0,9000003627287624,radio stereo alarm_clock
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle fence
3,9000038588854897,mountain tent The_Eiffel_Tower
4,9000052667981386,campfire fireplace leaf


(112199, 2)

In [24]:
model.save('seres34_512bat_128reso_full_highe')
#model = load_model('black-white-7.model', custom_objects={'top_3_accuracy': top_3_accuracy})

In [25]:
#Accuracy? 
valid_predictions1 = model.predict(x_valid, batch_size=128, verbose=1)
map3 = mapk(valid_df[['y']].values, preds2catids(valid_predictions1).values)
top3 = top3_acc(valid_df[['y']].values.flatten(), preds2catids(valid_predictions1).values)
print('Map3: {:.3f}'.format(map3))
print('Top3: {:.3f}'.format(top3))
print()

x_valid2 = np.array( [ np.fliplr(x_valid[i]) for i in range(x_valid.shape[0])] )
valid_predictions2 = model.predict(x_valid2, batch_size=128, verbose=1)
map3 = mapk(valid_df[['y']].values, preds2catids(valid_predictions2).values)
top3 = top3_acc(valid_df[['y']].values.flatten(), preds2catids(valid_predictions2).values)
print('Map3: {:.3f}'.format(map3))
print('Top3: {:.3f}'.format(top3))
print()

map3 = mapk(valid_df[['y']].values, preds2catids(0.5*valid_predictions1+0.5*valid_predictions2).values)
top3 = top3_acc(valid_df[['y']].values.flatten(), preds2catids(0.5*valid_predictions1+0.5*valid_predictions2).values)
print('Map3: {:.3f}'.format(map3))
print('Top3: {:.3f}'.format(top3))
print()

30000/30000 [==============================] - 40s 1ms/step
Map3: 0.858
Top3: 0.926

30000/30000 [==============================] - 40s 1ms/step
Map3: 0.857
Top3: 0.926

Map3: 0.860
Top3: 0.927

